In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)

In [2]:
# cd drive/MyDrive/IE7500_GroupB/Notebooks

In [3]:
!pip install transformers datasets

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# load necesary libraries
import numpy as np
import pandas as pd

In [5]:
dtypes_dict = {'headline': 'object',
               'url': 'object',
               'publisher': 'object',
               'stock': 'object',
               'tokens': 'object',
               'normalized_tokens': 'object',
               'filtered_tokens': 'object',
               'lemmas': 'object',
               'sentiment_score': 'float64',
               'Name': 'object',
               'Market Cap': 'float64',
               'Country': 'object',
               'IPO Year': 'float64',
               'Sector': 'object',
               'Industry': 'object',
               'year': 'int32',
               'month': 'int32',
               'day_of_week': 'int32',
               'sentiment_label': 'int64',
               'headline_length': 'int64',
               'word_count': 'int64',
               'Market_Cap_Category': 'object',
               'recommendation': 'object',
               'cap_Large': 'bool',
               'cap_Medium': 'bool',
               'cap_Mega': 'bool',
               'cap_Micro': 'bool',
               'cap_Nano': 'bool',
               'cap_Small': 'bool',
               'sector_Basic Materials': 'bool',
               'sector_Consumer Discretionary': 'bool',
               'sector_Consumer Staples': 'bool',
               'sector_Energy': 'bool',
               'sector_Finance': 'bool',
               'sector_Health Care': 'bool',
               'sector_Industrials': 'bool',
               'sector_Miscellaneous': 'bool',
               'sector_Real Estate': 'bool',
               'sector_Technology': 'bool',
               'sector_Telecommunications': 'bool',
               'sector_Utilities': 'bool', 'recommendation_label': 'int64',
               'publisher_label': 'int64', 'country_label': 'int64',
               'industry_label': 'int64'}

In [6]:
# load dataframes to use
df_main = pd.read_csv("saved_dfs/df_for_models.csv", dtype=dtypes_dict,
                      parse_dates=['date'])

In [7]:
df_main.head()

,headline,url,publisher,date,stock,tokens,normalized_tokens,filtered_tokens,lemmas,sentiment_score,...,sector_Industrials,sector_Miscellaneous,sector_Real Estate,sector_Technology,sector_Telecommunications,sector_Utilities,recommendation_label,publisher_label,country_label,industry_label
0,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01,A,"['Agilent', 'Technologies', 'Announces', 'Pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technologies', 'announces', 'pric...","['agilent', 'technology', 'announces', 'pricin...",0.0000,...,True,False,False,False,False,False,1,4,45,18
1,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18,A,"['Agilent', '(', 'A', ')', 'Gears', 'Up', 'for...","['agilent', 'a', 'gears', 'up', 'for', 'q2', '...","['agilent', 'gears', 'q2', 'earnings', 'cards']","['agilent', 'gear', 'q2', 'earnings', 'card']",0.0000,...,True,False,False,False,False,False,1,16,45,18
2,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15,A,"['J.P.', 'Morgan', 'Asset', 'Management', 'Ann...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...","['morgan', 'asset', 'management', 'announces',...",0.3612,...,True,False,False,False,False,False,1,4,45,18
3,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15,A,"['Pershing', 'Square', 'Capital', 'Management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...","['pershing', 'square', 'capital', 'management'...",0.0000,...,True,False,False,False,False,False,1,4,45,18
4,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12,A,"['Agilent', 'Awards', 'Trilogy', 'Sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'awards', 'trilogy', 'sciences', '...","['agilent', 'award', 'trilogy', 'science', 'go...",0.4588,...,True,False,False,False,False,False,1,4,45,18


Create Vectors from Headlines (Vectorization Models)

To generate vectors for the headline column, you can use different NLP vectorization techniques, including:

TF-IDF (Term Frequency - Inverse Document Frequency)

Count Vectorizer (Bag of Words)

Word2Vec (Pre-trained Word Embeddings)

Doc2Vec (Sentence-Level Embeddings)

BERT Embeddings (Transformer-based Representation)

# Goal 1: Predict Buy / Hold / Sell

## Method 4: MLP

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.utils import to_categorical
import numpy as np

2025-04-20 14:59:17.949588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745175557.969659 1314578 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745175557.975768 1314578 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745175557.991409 1314578 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745175557.991422 1314578 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745175557.991425 1314578 computation_placer.cc:177] computation placer alr

In [9]:
# Step 1: Preprocess Text + Target using HEADLINE
text_column = 'headline'
X_text = df_main[text_column].fillna('').astype(str)  # Clean headlines
y = df_main.loc[X_text.index, 'recommendation_label']

# Step 2: Train/test split
X_train_text, X_test_text, y_train_raw, y_test_raw = train_test_split(
    X_text, y, test_size=0.2, stratify=y, random_state=42
)

# Step 3: TF-IDF vectorization
vectorizer = TfidfVectorizer(max_features=1000)
X_train_vec = vectorizer.fit_transform(X_train_text)
X_test_vec = vectorizer.transform(X_test_text)

X_train_dense = X_train_vec.toarray().astype("float32")
X_test_dense = X_test_vec.toarray().astype("float32")

# Step 4: One-hot encode labels
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)
num_classes = y_train.shape[1]

In [10]:
# Define the model
model = Sequential([
    Dense(128, activation="relu", input_shape=(X_train_vec.shape[1],)),
    Dropout(0.5),
    Dense(64, activation="relu"),
    Dropout(0.5),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1745175605.401869 1314578 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11437 MB memory:  -> device: 0, name: Tesla P100-PCIE-12GB, pci bus id: 0000:04:00.0, compute capability: 6.0


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │       128,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 3)              │           195 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 136,579 (533.51 KB)

 Trainable params: 136,579 (533.51 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# Early stopping to reduce overfitting
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_dense, y_train,
    validation_data=(X_test_dense, y_test),
    epochs=5,
    batch_size=32,
    callbacks=[early_stop]
)

Epoch 1/5


I0000 00:00:1745175614.374946 1314712 service.cc:152] XLA service 0x14c0f00051d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745175614.374978 1314712 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-12GB, Compute Capability 6.0
2025-04-20 15:00:14.405627: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1745175614.582597 1314712 cuda_dnn.cc:529] Loaded cuDNN version 90300


  102/46139 ━━━━━━━━━━━━━━━━━━━━ 1:08 1ms/step - accuracy: 0.7570 - loss: 0.8353   

I0000 00:00:1745175615.915103 1314712 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


46139/46139 ━━━━━━━━━━━━━━━━━━━━ 86s 2ms/step - accuracy: 0.9238 - loss: 0.2601 - val_accuracy: 0.9449 - val_loss: 0.1920
Epoch 2/5
46139/46139 ━━━━━━━━━━━━━━━━━━━━ 79s 2ms/step - accuracy: 0.9410 - loss: 0.2054 - val_accuracy: 0.9479 - val_loss: 0.1827
Epoch 3/5
46139/46139 ━━━━━━━━━━━━━━━━━━━━ 79s 2ms/step - accuracy: 0.9439 - loss: 0.1977 - val_accuracy: 0.9486 - val_loss: 0.1801
Epoch 4/5
46139/46139 ━━━━━━━━━━━━━━━━━━━━ 80s 2ms/step - accuracy: 0.9455 - loss: 0.1937 - val_accuracy: 0.9494 - val_loss: 0.1780
Epoch 5/5
46139/46139 ━━━━━━━━━━━━━━━━━━━━ 82s 2ms/step - accuracy: 0.9457 - loss: 0.1927 - val_accuracy: 0.9512 - val_loss: 0.1739


In [12]:
# Evaluate and report
y_pred_probs = model.predict(X_test_dense)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

print("MLP Model Performance on Headlines:")
print(classification_report(y_true_classes, y_pred_classes, zero_division=0))

11535/11535 ━━━━━━━━━━━━━━━━━━━━ 11s 902us/step
MLP Model Performance on Headlines:
              precision    recall  f1-score   support

           0       0.94      0.72      0.82     36913
           1       0.95      0.99      0.97    317698
           2       0.94      0.62      0.75     14501

    accuracy                           0.95    369112
   macro avg       0.95      0.78      0.85    369112
weighted avg       0.95      0.95      0.95    369112



In [13]:
model.save("keras_mlp_recommendation_model.h5")
print("Model saved as keras_mlp_recommendation_model.h5")

Model saved as keras_mlp_recommendation_model.h5


## Method 5: LSTM for Text-Based Prediction

In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report

In [15]:
# Use raw headlines
texts = df_main["headline"].fillna("")
labels = df_main["recommendation_label"]

# Train/test split
X_train_text, X_test_text, y_train_raw, y_test_raw = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

In [16]:
# One-hot encode target
y_train = to_categorical(y_train_raw)
y_test = to_categorical(y_test_raw)
num_classes = y_train.shape[1]

In [17]:
# Tokenize text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(X_train_text)

# Convert to sequences
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
X_test_seq = tokenizer.texts_to_sequences(X_test_text)

# Pad sequences
max_len = 30
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding="post", truncating="post")
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding="post", truncating="post")


In [18]:

# Define LSTM model
embedding_dim = 100
model = Sequential([
    Embedding(input_dim=10000, output_dim=embedding_dim, input_length=max_len),
    LSTM(64, return_sequences=True),
    Dropout(0.3),
    LSTM(32),
    Dropout(0.3),
    Dense(32, activation="relu"),
    Dense(num_classes, activation="softmax")
])

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()

/courses/IE7500.202530/shared/conda_env_1/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [19]:
# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Train
history = model.fit(
    X_train_pad, y_train,
    validation_data=(X_test_pad, y_test),
    epochs=10,
    batch_size=512,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 42s 13ms/step - accuracy: 0.9171 - loss: 0.2817 - val_accuracy: 0.9822 - val_loss: 0.0616
Epoch 2/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9839 - loss: 0.0567 - val_accuracy: 0.9860 - val_loss: 0.0480
Epoch 3/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9878 - loss: 0.0412 - val_accuracy: 0.9883 - val_loss: 0.0415
Epoch 4/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9912 - loss: 0.0308 - val_accuracy: 0.9896 - val_loss: 0.0390
Epoch 5/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9933 - loss: 0.0241 - val_accuracy: 0.9903 - val_loss: 0.0384
Epoch 6/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9947 - loss: 0.0192 - val_accuracy: 0.9916 - val_loss: 0.0351
Epoch 7/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9958 - loss: 0.0154 - val_accuracy: 0.9914 - val_loss: 0.0366
Epoch 8/10
2884/2884 ━━━━━━━━━━━━━━━━━━━━ 37s 13ms/step - accuracy: 0.9964 -

In [21]:
# Predict
y_pred_probs = model.predict(X_test_pad)
y_pred_classes = np.argmax(y_pred_probs, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Report
print("LSTM Model Performance on Headlines:")
print(classification_report(y_true_classes, y_pred_classes, zero_division=0))


11535/11535 ━━━━━━━━━━━━━━━━━━━━ 23s 2ms/step
LSTM Model Performance on Headlines:
              precision    recall  f1-score   support

           0       0.98      0.96      0.97     36913
           1       0.99      1.00      1.00    317698
           2       0.97      0.95      0.96     14501

    accuracy                           0.99    369112
   macro avg       0.98      0.97      0.98    369112
weighted avg       0.99      0.99      0.99    369112



In [22]:
model.save("lstm_text_recommendation_model.h5")
print("Model saved as lstm_text_recommendation_model.h5")

Model saved as lstm_text_recommendation_model.h5


# Model 6: Custom Transformer Model for Financial News Classification

In [23]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LayerNormalization, Dense, Dropout, Embedding, Layer, Input
from tensorflow.keras.models import Model
from imblearn.over_sampling import RandomOverSampler
import re


In [24]:
class MultiHeadSelfAttention(Layer):
    def __init__(self, d_model, num_heads):
        super().__init__()
        assert d_model % num_heads == 0
        self.num_heads = num_heads
        self.depth = d_model // num_heads
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q):
        batch_size = tf.shape(q)[0]
        q = self.split_heads(self.wq(q), batch_size)
        k = self.split_heads(self.wk(k), batch_size)
        v = self.split_heads(self.wv(v), batch_size)
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        output = tf.transpose(output, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(output, (batch_size, -1, self.num_heads * self.depth))
        return self.dense(concat_attention)


In [25]:
class TransformerBlock(Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super().__init__()
        self.att = MultiHeadSelfAttention(d_model, num_heads)
        self.ffn = tf.keras.Sequential([
            Dense(dff, activation='relu'),
            Dense(d_model)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, x):
        attn_output = self.att(x, x, x)
        out1 = self.layernorm1(x + self.dropout1(attn_output))
        ffn_output = self.ffn(out1)
        return self.layernorm2(out1 + self.dropout2(ffn_output))

class TransformerModel(Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, num_classes, rate=0.1):
        super().__init__()
        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = self.positional_encoding(maximum_position_encoding, d_model)
        self.enc_layers = [TransformerBlock(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = Dropout(rate)
        self.global_avg_pool = tf.keras.layers.GlobalAveragePooling1D()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, x):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        for enc_layer in self.enc_layers:
            x = enc_layer(x)
        x = self.global_avg_pool(x)
        return self.classifier(x)

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(tf.range(position)[:, tf.newaxis], tf.range(d_model)[tf.newaxis, :], d_model)
        sines = tf.math.sin(angle_rads[:, 0::2])
        cosines = tf.math.cos(angle_rads[:, 1::2])
        return tf.cast(tf.concat([sines, cosines], axis=-1)[tf.newaxis, ...], tf.float32)

    def get_angles(self, pos, i, d_model):
        pos = tf.cast(pos, tf.float32)
        i = tf.cast(i, tf.float32)
        angle_rates = 1 / tf.pow(10000., (2 * (i // 2)) / tf.cast(d_model, tf.float32))
        return pos * angle_rates


In [26]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from imblearn.over_sampling import RandomOverSampler

# Split the dataset manually
train_df, test_df = train_test_split(df_main, test_size=0.2, stratify=df_main['recommendation_label'], random_state=42)

# Tokenize and pad
tokenizer = Tokenizer(num_words=10000, oov_token='<OOV>')
tokenizer.fit_on_texts(train_df['headline'])

X_train = tokenizer.texts_to_sequences(train_df['headline'])
X_test = tokenizer.texts_to_sequences(test_df['headline'])

X_train = pad_sequences(X_train, maxlen=50, padding='post')
X_test = pad_sequences(X_test, maxlen=50, padding='post')

y_train = train_df['recommendation_label'].values
y_test = test_df['recommendation_label'].values

# Oversample
ros = RandomOverSampler()
X_train_resampled, y_train_resampled = ros.fit_resample(X_train, y_train)

# Define and train transformer model
model = TransformerModel(
    num_layers=2,
    d_model=64,
    num_heads=4,
    dff=128,
    input_vocab_size=len(tokenizer.word_index) + 1,
    maximum_position_encoding=50,
    num_classes=3,  # update if different
    rate=0.1
)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train_resampled, y_train_resampled, epochs=10, batch_size=32, validation_split=0.1)

# Evaluate
loss, acc = model.evaluate(X_test, y_test)
print(f"Transformer Test Accuracy: {acc:.4f}")


Epoch 1/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 466s 4ms/step - accuracy: 0.9458 - loss: 0.1437 - val_accuracy: 0.9812 - val_loss: 0.0520
Epoch 2/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 460s 4ms/step - accuracy: 0.9868 - loss: 0.0418 - val_accuracy: 0.9971 - val_loss: 0.0122
Epoch 3/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 456s 4ms/step - accuracy: 0.9913 - loss: 0.0289 - val_accuracy: 0.9944 - val_loss: 0.0167
Epoch 4/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 474s 4ms/step - accuracy: 0.9925 - loss: 0.0254 - val_accuracy: 0.9974 - val_loss: 0.0100
Epoch 5/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 465s 4ms/step - accuracy: 0.9927 - loss: 0.0250 - val_accuracy: 0.9986 - val_loss: 0.0052
Epoch 6/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 505s 5ms/step - accuracy: 0.9929 - loss: 0.0240 - val_accuracy: 0.9943 - val_loss: 0.0196
Epoch 7/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 491s 5ms/step - accuracy: 0.9929 - loss: 0.0241 - val_accuracy: 0.9964 - val_loss: 0.0142
Epoch 8/10
107223/107223 ━━━━━━━━━━━━━━━━━━━━ 51